This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

# Load the results + get generated suffixes

In [3]:
# dir_single_query = '/home/chansingh/mntv1/single_query_math_9_17' # single-digit
# dir_single_query = '/home/chansingh/mntv1/single_query_math_double_digit'
# dir_single_query = '/home/chansingh/mntv1/single_query_math_long_suffs' # suffix results
dir_single_query = '/home/chansingh/mntv1/suffix_math_long_suffs_full' # suffix results
# r = analyze_utils.load_results_and_cache(dir_single_query, save_file='r_long.pkl', only_keep_scalar=False)
r_single_query = pd.read_pickle(os.path.join(dir_single_query, 'r_long.pkl'))
r_single_query = analyze_utils.postprocess_results(r_single_query)

# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_9_17' # single-digit
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_double_digit'
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_long_suffs'
dir_suffix_search = '/home/chansingh/mntv1/suffix_anli_long_suffs_full'
# r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r_long.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r_long.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search)

# r = r_single_query
r = pd.concat((r_single_query, r_suffix_search)).sort_values('checkpoint')
# r.to_pickle(f'../results/prompt_gen/results_suffix_math.pkl')

# Look at top suffix candidates

In [46]:
r[r.max_num_tokens == 6]

,task_name,n_shots,max_dset_size,max_digit,template_num_init_string,template_num_task_phrasing,checkpoint,max_num_tokens,beam_size,beam_size_extra,...,Recall @ 30 suffixes,Recall @ 40 suffixes,Recall @ 50 suffixes,Recall @ 75 suffixes,Recall @ 100 suffixes,Recall @ 150 suffixes,Recall @ 200 suffixes,reciprocal_rank,reciprocal_rank_multi,train_split_frac
2,task1146_country_capital,5,1000,10,0,0,EleutherAI/gpt-j-6B,6,5,50,...,False,False,False,False,False,False,True,0.005747,0.015152,1
17,task1146_country_capital,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.001808,0.001845,1
30,task1147_country_currency,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,True,0.005102,0.015385,1
5,task1509_evalution_antonyms,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.001031,0.002410,1
1,multiply_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.002326,0.012346,1
8,max_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,True,0.006452,0.011905,1
12,subtract_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,True,True,0.007143,0.014286,1
14,divide_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.004329,0.004854,1
19,first_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,True,True,True,True,True,0.020408,0.020000,1
26,double_one,1,1000,10,0,0,gpt2-xl,6,5,50,...,True,True,True,True,True,True,True,0.090909,0.083333,1


In [ ]:
r[['suffix_str_added', 'final_answer_added']].values

In [25]:
args = r.iloc[1]



In [4]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1)]
for i in range(r0.shape[0]):
    row = r0.iloc[i]
    print('task', row.task_name)
    candidates, probs = analyze_utils.get_top_candidates_and_probs_suff(row)
    for j in range(5):
        print('\t', candidates[j], probs[j])

task task1146_country_capital
	  the name of the 0.027337204643153114
	  the value of the 0.010155583117537137
	  the string representation of 0.0034278519205488264
	  the first letter of 0.003237949962753727
	  the first word of 0.0024385488001707474
task subtract_two
	  the answer from the input 0.001433538474841672
	  the number from the input 0.0014024320485071292
	  the input number to the 0.0012383299494923923
	  the value of the input 0.0012312262495509424
	  the answer to the input 0.0008505216155874579
task square_one
	  the answer to the input 0.001288408550999658
	  the value of the input 0.0007730979235454073
	  the output of the function 0.0007226405868898953
	  the answer from the output 0.000627642959796317
	  the input to the function 0.0005471376976459741
task task1149_item_check_edible


KeyboardInterrupt: 

# Actually test acc

In [37]:
r.task_name

1     task092_check_prime_classification
21                          subtract_two
7               task1191_food_veg_nonveg
6            task1509_evalution_antonyms
17                               exp_one
                     ...                
19                             first_two
18                         fibonacci_one
33                         fibonacci_one
24                               add_two
36        task107_splash_question_to_sql
Name: task_name, Length: 77, dtype: object

In [38]:
args = r.iloc[3]

In [39]:
np.random.seed(args.seed)
(dset, dset_test), check_answer_func, descr = data.get_data(
            args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)

In [40]:
args.task_name

'task1509_evalution_antonyms'

In [41]:
dset[0]

{'input': 'Input: fact Answer:',
 'output': ' lie\n\n',
 'text': 'Input: fact Answer: lie\n',
 '__index_level_0__': 160}

In [43]:
checkpoint = 'EleutherAI/gpt-j-6B' # 'gpt2-medium'
model = prompt_classification.create_model(checkpoint)

In [46]:
# todo: work with proper padding
# todo: work with multitoken outputs
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=f'Return the antonym of the input: ', multi_token=True,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


x_text[0]:'Subtract these numbers: Input: fact Answer:'
y_text[0]:' lie\n\n'


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0
	x_text 'Subtract these numbers: Input: fact Answer:'
	new_text ' fact\n\nSubtract these'
	y_text ' lie\n\n'
	y_pred ' lie'
1
	x_text 'Subtract these numbers: Input: lemon Answer:'
	new_text ' 2\n\nSubtract these'
	y_text ' orange\n\n'
	y_pred ' orange'
2
	x_text 'Subtract these numbers: Input: committee Answer:'
	new_text ' committee\n\nSubtract these'
	y_text ' individual\n\n'
	y_pred ' individual'
3
	x_text 'Subtract these numbers: Input: polish Answer:'
	new_text ' 1\n\nSubtract these'
	y_text ' rough\n\n'
	y_pred ' rough'
4
	x_text 'Subtract these numbers: Input: dry Answer:'
	new_text ' wet\n\nSubtract these'
	y_text ' wet\n\n'
	y_pred ' wet'
5
	x_text 'Subtract these numbers: Input: girlfriend Answer:'
	new_text ' 0\n\nSubtract these'
	y_text ' enemy\n\n'
	y_pred ' enemy'
6
	x_text 'Subtract these numbers: Input: lift Answer:'
	new_text ' 0.0\n\nInput:'
	y_text ' set\n\n'
	y_pred ' set'
7
	x_text 'Subtract these numbers: Input: large Answer:'
	new_text ' small\n\nInput: small 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0
	x_text 'Subtract these numbers: Input: far Answer:'
	new_text ' near\n\nSubtract these'
	y_text ' nearby\n\n'
	y_pred ' nearby'
1
	x_text 'Subtract these numbers: Input: lady Answer:'
	new_text ' lady\n\nSubtract these'
	y_text ' man\n\n'
	y_pred ' man'
2
	x_text 'Subtract these numbers: Input: girl Answer:'
	new_text ' girl\n\nSubtract these'
	y_text ' son\n\n'
	y_pred ' son'
3
	x_text 'Subtract these numbers: Input: free Answer:'
	new_text ' 0\n\nSubtract these'
	y_text ' trap\n\n'
	y_pred ' trap'
4
	x_text 'Subtract these numbers: Input: enlightenment Answer:'
	new_text ' 0\n\nSubtract these'
	y_text ' ignorance\n\n'
	y_pred ' ignorance'
5
	x_text 'Subtract these numbers: Input: reward Answer:'
	new_text ' 0\n\nInput: reward Answer'
	y_text ' stick\n\n'
	y_pred ' stick'
6
	x_text 'Subtract these numbers: Input: speak Answer:'
	new_text ' speak\nInput: read Answer:'
	y_text ' write\n\n'
	y_pred ' write'
7
	x_text 'Subtract these numbers: Input: ride Answer:'
	new_text ' 0\n\nSubtr

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0
	x_text 'Subtract these numbers: Input: sit Answer:'
	new_text '<|endoftext|>Q\n\nSubtract these'
	y_text ' stand\n\n'
	y_pred ' stand'
1
	x_text 'Subtract these numbers: Input: point Answer:'
	new_text '<|endoftext|>Q\n\nSubtract these'
	y_text ' flat\n\n'
	y_pred ' flat'
2
	x_text 'Subtract these numbers: Input: happy Answer:'
	new_text '<|endoftext|>QQQQQQQ'
	y_text ' unpleasant\n\n'
	y_pred ' unpleasant'
3
	x_text 'Subtract these numbers: Input: pay Answer:'
	new_text '<|endoftext|>Q: What is the difference between'
	y_text ' tax\n\n'
	y_pred ' tax'
4
	x_text 'Subtract these numbers: Input: join Answer:'
	new_text '<|endoftext|>QQQQQQQ'
	y_text ' split\n\n'
	y_pred ' split'
5
	x_text 'Subtract these numbers: Input: mild Answer:'
	new_text '<|endoftext|>Q: What is the difference between'
	y_text ' violent\n\n'
	y_pred ' violent'
6
	x_text 'Subtract these numbers: Input: happiness Answer:'
	new_text '<|endoftext|>Q: What is the difference between'
	y_text ' unhappiness\n\n'
	y_pred

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
